In [2]:
from opt.mc_sim import *
from common.variables import *
from sim.sim_functions import *
import torch
import gym
from gym import spaces
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, Tuple, MultiBinary
import tensorflow as tf; print(tf.__version__)

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-28 08:22:33.380322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 08:22:34.023746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-28 08:22:34.023808: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:

2.11.0


In [3]:
s = SourcingEnv()

In [4]:
class CustomGymEnv(Env):
        
    def __init__(self, sourcing_env):
        self.SourcingEnv = sourcing_env
        self.counter = 0
        
        # Actions we can take, down, stay, up
        self.action_space = MultiDiscrete([INVEN_LIMIT,INVEN_LIMIT])

        # Inventory Observation State
        self.observation_space = Box(low=np.array([-30, 0, 0, 0, 0]), high=np.array([30, 30, 30, 1, 1]), shape=(5,), dtype=int)
                                               
    
    def step(self, action):        
        reward = self.reward_func(self.SourcingEnv.current_state, action)
        next_state, event, i, event_probs, supplier_index = self.SourcingEnv.step(action)
        self.counter += 1
        
        info = {}
        
        if self.counter < PERIODS:
            done = False
        else:
            done = True
        
        next_state_array = np.array(next_state.get_list_repr())
        return next_state_array, reward, done, info
    
    def reset(self):
        self.SourcingEnv = SourcingEnv()
        return np.array(self.SourcingEnv.current_state.get_list_repr())
        
    def reward_func(self, state, action):
        reward_hb = H_COST * state.s if state.s >= 0 else (-B_PENALTY * state.s )
        reward = reward_hb + np.sum(np.multiply(action, PROCUREMENT_COST_VEC))
        reward = float(reward)
        return -reward

In [5]:
#INVEN_LIMIT

In [6]:
#Discrete(INVEN_LIMIT)

In [7]:
s = SourcingEnv()

In [8]:
custom_gym_env = CustomGymEnv(s)

In [9]:
m_state = custom_gym_env.SourcingEnv.current_state

str(m_state)

'Stock: 0, n backorders: [0. 0.], supplier status (on/off): [1. 1.]'

In [10]:
#PERIODS

In [11]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([2, 2]))

In [12]:
str(next_state_array)

'[-1  2  2  1  1]'

In [13]:
reward

-100.0

In [14]:
custom_gym_env.reset()

array([0, 0, 0, 1, 1])

In [15]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [16]:
str(next_state_array)

'[-1  0  0  1  1]'

In [17]:
reward

-0.0

In [18]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [19]:
str(next_state_array)

'[-2  0  0  1  1]'

In [20]:
reward

-10.0

In [21]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [22]:
str(next_state_array)

'[-3  0  0  1  1]'

In [23]:
reward

-20.0

In [24]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [25]:
str(next_state_array)

'[-4  0  0  1  1]'

In [26]:
reward

-30.0

In [27]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [28]:
str(next_state_array)

'[-5  0  0  1  1]'

In [29]:
reward

-40.0

In [30]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [31]:
str(next_state_array)

'[-6  0  0  1  1]'

In [32]:
reward

-50.0

In [33]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [34]:
str(next_state_array)

'[-7  0  0  1  1]'

In [35]:
reward

-60.0

In [36]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))


In [37]:
str(next_state_array)

'[-7  0  0  1  0]'

In [38]:
reward

-70.0

In [39]:
from stable_baselines3.common.env_checker import check_env

In [40]:
check_env(custom_gym_env) #warn=True)

In [41]:
episodes = 10
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    
    
    while not done:
        
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost=reward
        observation = custom_gym_env.step(action)
    print('Episode:{} Cost:{} Observation:{}' .format(episode, cost, observation))   #Observation: {}(, observation)
custom_gym_env.close()

Episode:1 Cost:-970.0 Observation:(array([20, 20, 28,  1,  1]), -970.0, True, {})
Episode:2 Cost:-450.0 Observation:(array([-1, 14, 54,  1,  1]), -450.0, True, {})
Episode:3 Cost:-675.0 Observation:(array([12, 12, 54,  1,  1]), -675.0, True, {})
Episode:4 Cost:-230.0 Observation:(array([ 3,  3, 38,  1,  1]), -230.0, True, {})
Episode:5 Cost:-810.0 Observation:(array([27, 30, 27,  1,  1]), -810.0, True, {})
Episode:6 Cost:-320.0 Observation:(array([7, 7, 2, 1, 1]), -320.0, True, {})
Episode:7 Cost:-145.0 Observation:(array([20,  2, 20,  1,  1]), -145.0, True, {})
Episode:8 Cost:-375.0 Observation:(array([ 7,  7, 24,  1,  1]), -375.0, True, {})
Episode:9 Cost:-1040.0 Observation:(array([10, 44, 10,  1,  1]), -1040.0, True, {})
Episode:10 Cost:-985.0 Observation:(array([20, 20, 34,  1,  1]), -985.0, True, {})


In [42]:
from stable_baselines3 import A2C, DDPG, DQN, PPO, TD3, SAC
import os



In [43]:
env = custom_gym_env

In [44]:
log_path = os.path.join('A2C', 'Training', 'Logs')

In [45]:
from stable_baselines3 import A2C
policy_kwargs = dict(net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
model = A2C("MlpPolicy", env, verbose=1,  learning_rate=(0.001),\
 n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \
tensorboard_log=log_path, seed=50, device='cuda' )
model.learn(total_timesteps=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to A2C/Training/Logs/A2C_1


In [46]:
model.learn(total_timesteps=10)

Logging to A2C/Training/Logs/A2C_2


In [47]:
#import gym
#from stable_baselines3 import A2C
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
 #                    net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
#model = A2C("MlpPolicy", env, verbose=1,  learning_rate=linear_schedule(0.001),\
 #n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \
#tensorboard_log=logdir,  policy_kwargs=policy_kwargs, seed=50, device='cuda' )
#model.learn(total_timesteps=50000)

In [48]:
model.save('A2C')

In [49]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-390.0, 0.0)

In [50]:
custom_gym_env.observation_space

Box([-30   0   0   0   0], [30 30 30  1  1], (5,), int64)

In [51]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [52]:
log_path = os.path.join('PPO','Training', 'Logs')

In [53]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.01, device='cuda')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [54]:
model.learn(total_timesteps=10000)

Logging to PPO/Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -781     |
| time/              |          |
|    fps             | 1194     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -626      |
| time/                   |           |
|    fps                  | 971       |
|    iterations           | 2         |
|    time_elapsed         | 4         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0814244 |
|    clip_fraction        | 0.888     |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.73     |
|    explained_variance   | 1.19e-07  |
|    learning_rate       

In [55]:
model.save('PPO')

In [56]:
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

(-75.0, 0.0)

In [57]:
import torch

In [58]:

torch.cuda.is_available()

False

In [59]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [60]:
 torch.cuda.device_count()

0